In [ ]:
import numpy as np
import os, pdb, sys
os.environ['CUDA_VISIBLE_DEVICES'] = '0'
sys.path.append('.')
import torch
torch.backends.cuda.matmul.allow_tf32 = True
torch.backends.cudnn.allow_tf32 = True
torch.backends.cudnn.deterministic = True
import diffuser.utils as utils
from diffuser.models import GaussianDiffusionPB
from diffuser.guides.policies_compose import PolicyCompose
from diffuser.utils.comp.comp_setup import ComposedParser
from diffuser.utils.jupyter_utils import suppress_stdout
from tap import Tap
import argparse

## You need to download 'rm2d-comp' from OneDrive Link in README.md to launch
config = "config/comp/Comp_rSmaze_nw6nw3_hExt0505gsd_engy_add.py"
class Parser(Tap):
    config: str = config
    plan_n_maze: int = 1
    
#---------------------------------- setup ----------------------------------#
from diffuser.guides.comp_plan_env import ComposedEnvPlanner
from diffuser.guides.comp_plan_helper import ComposedDiffusionPlanner
sys.argv = ['PlaceHolder.py', '--config', config, ]

with suppress_stdout():
    args_comp = Parser().parse_args()
    args_comp.cond_w = [2.0, 2.0]
    cp = ComposedParser(args_comp.config, args_cmd=args_comp)
    args_train_list, args_list = cp.setup_args_list(args_comp, )
    dplanner = ComposedDiffusionPlanner(args_train_list, args_list, cp.comp_dataset)

plan_env_list = dplanner.plan_env_list

#### Create an Env

In [ ]:
from diffuser.utils.rm2d_render import RandStaticMazeRenderer
import imageio
from importlib import reload
np.set_printoptions(precision=3)
# import pb_diff_envs.environment.static.rand_maze2d_renderer as rmr; reload(rmr)
from pb_diff_envs.environment.static.rand_maze2d_renderer import RandMaze2DRenderer
rootdir = './visualization/'

bs = 50 # batch size

env_source = 'dataset' # or 'user_input'
if env_source == 'user_input':
    env_id = 0 # placeholder
    wall_locations_c = [
        np.array([[0.805, 3.83 ],
                [0.852, 1.6  ],
                [2.474, 2.591],
                [2.585, 1.248],
                [4.116, 3.365],
                [4.128, 2.096],]),
        np.array([
                [0.813, 3.648],
                [1.617, 2.137],
                [2.947, 2.581],
                [2.956, 0.718],
                [4.347, 3.962]]),
    ]
    ## size of the obstacles 1x1 square block
    if len(wall_locations_c) == 2:
        hExt_list_c = [np.array([[0.5, 0.5],]*6), np.array([[0.5, 0.5],] * len(wall_locations_c[1]) )]
    else:
        hExt_list_c = [np.array([[0.5, 0.5],]*6), ]
    env = plan_env_list.create_env_by_pos(env_id, wall_locations_c, hExt_list_c)

elif env_source == 'dataset':
    env_id = 1 # input an index
    env = plan_env_list.create_single_env(env_id)


obs_start_np = np.array( [ [ 0.22, 4.53 ] ], dtype=np.float32 ).repeat( bs, 0 )
target_np = np.array( [ [3.93, 0.60] ], dtype=np.float32  ).repeat( bs, 0 )

e_wlocs = env.wall_locations
hExt = plan_env_list.hExt_list[0, 0].tolist() ## [0.5, 0.5]
env_hExts = np.array([hExt,] * len(e_wlocs)) ## dupliate the wloc
print(f'[Training Time] # of walls: 6')
print(f'[Now] # of walls: {len(e_wlocs)}')

mz_renderer = RandMaze2DRenderer(num_walls_c = list(map(lambda x: len(env.wloc_list_c[x]), list(range(len(env.wloc_list_c))) )), fig_dpi=200)
mz_renderer.up_right = 'empty'
mz_renderer.config['no_draw_traj_line'] = True
mz_renderer.config['no_draw_col_info'] = True

img = mz_renderer.renders( np.concatenate([obs_start_np[0:1,], target_np[0:1,],]), plan_env_list.maze_size, e_wlocs, env_hExts )
utils.plt_img(img, dpi=80)

In [ ]:
env.wall_locations.shape ## (n_walls, 2)

### First Run env creation several blocks below

In [ ]:
import diffuser.guides.comp_plan_env as dcpe; reload(dcpe)
from diffuser.guides.comp_plan_env import ComposedEnvPlanner, ComposedRM2DReplanner

rm2d_planner = ComposedEnvPlanner()
policy: PolicyCompose = dplanner.policy
policy.ddim_eta = 1.0
use_ddim = True
repl_dn_steps = 5
policy.return_diffusion = True
print(f'policy guidance weight: {policy.cg_w}')
print(f'policy ddim_steps: {policy.ddim_steps}')

wloc_np = env.wall_locations ## (num_walls, 2)
repl_config = dict(use_ddim=use_ddim, dn_steps=repl_dn_steps)
replanner = ComposedRM2DReplanner(policy, dplanner.train_normalizer_list[0], repl_config, 'cuda')

obs_start_tensor = utils.to_torch(obs_start_np,  device='cpu')
target_tensor = utils.to_torch( target_np, device='cpu' )
wloc_input = wloc_np[None, ].repeat(bs, axis=0) # (10, 2) -> (bs, 10, 2)
wloc_input = wloc_input.reshape(bs, -1)
wall_locations = utils.to_torch( wloc_input,  device='cpu' )

use_normed_wallLoc = False
plan_env_config = utils.Config(None,
    env_mazelist=dplanner.plan_env_list,
    prob_permaze=1,
    samples_perprob=bs,
    obs_selected_dim=(0,1),
    horizon_pl_list=[40, 48, 52, 56, 60, 64],
    horizon_wtrajs=48,
    do_replan=False,
    replanner=replanner,
)
with suppress_stdout():
    ## return motion candidates of all horizon
    ## the motion plans: samples_mulho[i].observations (trajectories of states)
    _, samples_mulho, _ = rm2d_planner.plan_env_interact(
        env, policy, obs_start_tensor, target_tensor, wall_locations, use_normed_wallLoc, plan_env_config,
    )

In [ ]:
from diffuser.utils.jupyter_utils import get_all_suc_trajs
all_candidates = [] ## candidates of several horizons
for trajs in samples_mulho:
    all_candidates.extend(trajs.observations)

## pick out successful trajectories
print('len all_candidates:', len(all_candidates))
env.robot.collision_eps = 1e-2
suc_trajs, suc_idxs = get_all_suc_trajs(env, trajs=all_candidates, goal=target_np)

#### Render Motion Plans

In [ ]:
## Rendering
from diffuser.utils.rm2d_render import RandStaticMazeRenderer

rd = RandStaticMazeRenderer(plan_env_list)

tmp_path = f'{rootdir}/luotest.png'
renderer = plan_env_list.model_list[env_id].renderer
renderer.fig_dpi = 40 # lower resolution for fast render
renderer.up_right = 'default'
renderer.config['no_draw_traj_line'] = False
renderer.config['no_draw_col_info'] = False

rd.composite( tmp_path, suc_trajs[:10],  np.array([env_id])) # visualize suc trajs
img = imageio.imread(tmp_path)
utils.plt_img(img, dpi=200) # 600

### create a denoising video

In [ ]:
traj_idx = suc_idxs[0] # which path
hor_idx = traj_idx // bs
path_idx = traj_idx % bs # which path

## multi_hor_x0_perstep: stores all the x0 in the denoising process
n_steps = len(policy.multi_hor_x0_perstep[0]) ## number of denosing steps
trajs_ps = []
imgs_ps = []

mz_renderer = RandMaze2DRenderer(num_walls_c = list(map(lambda x: len(env.wloc_list_c[x]), list(range(len(env.wloc_list_c))) )), fig_dpi=200)
mz_renderer.up_right = 'default'
mz_renderer.config['no_draw_traj_line'] = False
mz_renderer.config['no_draw_col_info'] = False

for i_st in range(n_steps):
    traj_ps = policy.multi_hor_x0_perstep[hor_idx][i_st].observations[path_idx]
    trajs_ps.append(traj_ps)
    ### -----
    e_wlocs = env.wall_locations
    hExt = plan_env_list.hExt_list[0, 0].tolist()
    env_hExts = np.concatenate(env.hExt_list_c, axis=0)
    img = mz_renderer.renders( traj_ps , plan_env_list.maze_size, e_wlocs, env_hExts )
    imgs_ps.append(img)

In [ ]:
print('rootdir:', rootdir)
import diffuser.utils.video as duv; reload(duv)
from diffuser.utils.video import save_images_to_mp4, read_mp4_to_numpy

mp4_name = f'{rootdir}/comp_maze2d_env{env_id}.mp4' # {utils.get_time()}
## Save to video
save_images_to_mp4(imgs_ps, mp4_name, fps=15, st_sec=1, end_sec=1)
## load and save
import mediapy as media
from mediapy import show_video, read_video
show_video(read_mp4_to_numpy(mp4_name), fps=15, width=400)


In [ ]:
n_steps